In [1]:
import re
import csv
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
#import nltk
from collections import Counter 
import matplotlib.pyplot as plt

# Lecture des données et construction des train, dev et test sets

In [2]:
def clean_str(string, tolower=True):
    """
    Tokenization/string cleaning.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    if tolower:
        string = string.lower()
    return string.strip()

In [3]:
##On transforme les 3 colonnes en 1 : url, titre et le texte deviennent un seul élément
def read_data(filename):
    data = []
    labels = []
    with open(filename, encoding="utf8", newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            txt = clean_str(row[2])
            txt = " ".join(txt.split("\n"))
#             d = clean_str(row[0])+" "+clean_str(row[1])
            labels.append(row[3])
            data.append(txt)    
    data.pop(0)
    labels.pop(0)
    labels = [float(l) for l in labels]
    return np.array(data), np.array(labels)

In [4]:
data, labels = read_data("data.csv")

In [5]:
print(data[0])

image copyright getty images on sunday morning , donald trump went off on a twitter tirade against a member of his own party this , in itself , is n't exactly huge news it 's far from the first time the president has turned his rhetorical cannons on his own ranks this time , however , his attacks were particularly biting and personal he essentially called tennessee senator bob corker , the chair of the powerful senate foreign relations committee , a coward for not running for re election he said mr corker begged for the president 's endorsement , which he refused to give he wrongly claimed that mr corker 's support of the iranian nuclear agreement was his only political accomplishment unlike some of his colleagues , mr corker free from having to worry about his immediate political future did n't hold his tongue skip twitter post by senbobcorker it 's a shame the white house has become an adult day care center someone obviously missed their shift this morning senator bob corker \( senbo

In [6]:
print(data.shape)
fake_data = data[labels==0]
real_data = data[labels==1]
fake_labels = labels[labels==0]
real_labels = labels[labels==1]
print("Nombre de fake news : {}".format(len(fake_data)))
print("Nombre de news correctes : {}".format(len(real_data)))


(4009,)
Nombre de fake news : 2137
Nombre de news correctes : 1872


In [7]:
print(fake_data.shape)

(2137,)


In [8]:
train_data = np.array([*real_data[:int(len(real_data)*0.8)],*fake_data[:int(len(fake_data)*0.8)]])
train_labels = np.array([*real_labels[:int(len(real_labels)*0.8)],*fake_labels[:int(len(fake_labels)*0.8)]], dtype=np.float32)

dev_data = np.array([*real_data[int(len(real_data)*0.8):int(len(real_data)*0.9)], *fake_data[int(len(fake_data)*0.8):int(len(fake_data)*0.9)]])
dev_labels = np.array([*real_labels[int(len(real_labels)*0.8):int(len(real_labels)*0.9)], *fake_labels[int(len(fake_labels)*0.8):int(len(fake_labels)*0.9)]], dtype=np.float32)

test_data = np.array([*real_data[int(len(real_data)*0.9):], *fake_data[int(len(fake_data)*0.9):]])
test_labels = np.array([*real_labels[int(len(real_labels)*0.9):], *fake_labels[int(len(fake_labels)*0.9):]], dtype=np.float32)

In [9]:
print("taille du train set : {}".format(len(train_data)))
print("taille du dev set : {}".format(len(dev_data)))
print("taille du test set : {}".format(len(test_data)))

taille du train set : 3206
taille du dev set : 401
taille du test set : 402


# Construction du vocabulaire

In [10]:
txt = [w for txt in train_data for w in txt.split()]
          
# token = nltk.word_tokenize(txt)
words = Counter(txt)
words = sorted(words, key=words.get, reverse=True)
words = ['_PAD','_UNK'] + words
vocab_size = len(words)
word2idx = {o:i for i,o in enumerate(words)}
word2idx["UNK"]=len(word2idx)
print("vocab size : ",vocab_size)

vocab size :  43464


In [11]:
train_data = [[word2idx[w] for w in news.split()] for news in train_data]
dev_data = [[word2idx[w] if w in word2idx else 0 for w in news.split()] for news in dev_data]
test_data = [[word2idx[w] if w in word2idx else 0 for w in news.split()] for news in test_data]

In [12]:
def pad_input(data, seq_len):
    features = np.zeros((len(data), seq_len), dtype=int)
    for ii, review in enumerate(data):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

# Padding
## on pad les input pour pouvoir créer des datasets pytorch et simplifier l'apprentissage (fonction de train plus claire, batch plus facile, etc)
on peut tester plusieurs taille de padding : input le plus grand, moyen, min, ou arbitraire du genre 200, 500, etc. On peut aussi ainsi réduire le temps de train.

In [13]:
# train_seq_len = max([len(s) for s in train_data])
# dev_seq_len = max([len(s) for s in dev_data])
# test_seq_len = max([len(s) for s in test_data])

# train_data = pad_input(train_data, train_seq_len)
# dev_data = pad_input(dev_data, dev_seq_len)
# test_data = pad_input(test_data, test_seq_len)

seq_len = 200


train_data = pad_input(train_data, seq_len)
dev_data = pad_input(dev_data, seq_len)
test_data = pad_input(test_data, seq_len)

In [14]:
train_data = TensorDataset(th.from_numpy(train_data).type(th.LongTensor), th.from_numpy(train_labels))
dev_data = TensorDataset(th.from_numpy(dev_data).type(th.LongTensor), th.from_numpy(dev_labels))
test_data = TensorDataset(th.from_numpy(test_data).type(th.LongTensor), th.from_numpy(test_labels))

batch_size = 50

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
dev_loader = DataLoader(dev_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

# CBOW Classifieur

In [15]:
class L_CBOW_classifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(L_CBOW_classifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.l1 = nn.Linear(embedding_dim, 1)
        
        nn.init.xavier_uniform_(self.l1.weight.data)  # Xavier/Glorot init for tanh
        nn.init.zeros_(self.l1.bias.data)  # Xavier/Glorot init for tanh
        
    def forward(self, inputs):
        inputs = self.embedding(inputs)
        inputs = th.sum(inputs, dim=1)
        out = th.sigmoid(self.l1(inputs))
        return out

In [16]:
class C_CBOW_classifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(C_CBOW_classifier, self).__init__()
        self.embedding_dim = embedding_dim
        self.vocab_size = vocab_size
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.conv1 = nn.Conv1d(1, 20, 5, 1)
        self.conv2 = nn.Conv1d(20, 50, 5, 1)
        self.dense1 = nn.Linear(450, 250)
        self.dense2 = nn.Linear(250, 64)
        self.dense3 = nn.Linear(64, 1)

    def forward(self, x):

        x = self.embeddings(x)
        x = th.sum(x, dim=0)
        x = x.view(1, 1,-1)
        x = F.relu(self.conv1(x))
        x = F.max_pool1d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool1d(x, 2, 2)
        x = x.view(-1, 450)
        x = F.relu(self.dense1(x))
        x = F.relu(self.dense2(x))
        x = self.dense3(x)
        return th.sigmoid(x)

In [17]:
loss_fn = nn.BCELoss()
lr = 1e-2
l_cbow = L_CBOW_classifier(len(word2idx), 50)
cbow_train_accuracies = []
cbow_train_losses = []
cbow_dev_accuracies = []
cbow_dev_losses = []

In [18]:
def train_cbow(model, max_epochs=20):
    optim = th.optim.Adam(params=model.parameters(), lr =lr, weight_decay=1e-4)
#     model.train()
    
    for e in range(max_epochs):
        train_accuracy = 0
        dev_accuracy = 0
        train_mean_loss = 0
        dev_mean_loss = 0
        
        n=0
        for x, labels in train_loader:
            n+=batch_size
            preds = model(x)
            loss = loss_fn(preds, labels)

            train_mean_loss+=loss.item()
            optim.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optim.step()
            
            for pred, label in zip(preds, labels):
                if((pred<0.5 and label==0) or (pred>0.5 and label == 1)):
                    train_accuracy+=1
                
            if(n%250==0):
                print("step : {}/{} ".format(n, len(train_data)))
                print("step accuracy : ", train_accuracy/n)
                print("step loss : ", train_mean_loss/n)
                
        ###Dev test
        for x, labels in dev_loader:
            preds = model(x)
            loss = loss_fn(preds, labels)
            dev_mean_loss+=loss.item()
            for pred, label in zip(preds, labels):
                if((pred<0.5 and label==0) or (pred>0.5 and label == 1)):
                    dev_accuracy+=1
                    
        cbow_train_accuracies.append(train_accuracy/len(train_data))
        cbow_train_losses.append(train_mean_loss/len(train_data))
        cbow_dev_accuracies.append(dev_accuracy/len(dev_data))
        cbow_dev_losses.append(dev_mean_loss/len(dev_data))
        
        
        print("EPOCH {}".format(e+1))
        print("Train Accuracy : ",train_accuracy/len(train_data))
        print("Dev Accuracy : ",dev_accuracy/len(dev_data))
        print("Train Mean loss : ",train_mean_loss/len(train_data))
        print("Dev Mean loss : ",dev_mean_loss/len(dev_data))
        print("----------------------------------------")

In [19]:
train_cbow(l_cbow)

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


step : 250/3206 
step accuracy :  0.564
step loss :  0.21165738677978516
step : 500/3206 
step accuracy :  0.56
step loss :  0.20272714138031006
step : 750/3206 
step accuracy :  0.592
step loss :  0.17276039695739745
step : 1000/3206 
step accuracy :  0.596
step loss :  0.16031105518341066
step : 1250/3206 
step accuracy :  0.6192
step loss :  0.14377160167694092
step : 1500/3206 
step accuracy :  0.636
step loss :  0.13358170811335246
step : 1750/3206 
step accuracy :  0.656
step loss :  0.12726097842625209
step : 2000/3206 
step accuracy :  0.671
step loss :  0.12204022449254989
step : 2250/3206 
step accuracy :  0.6844444444444444
step loss :  0.11533693742752076
step : 2500/3206 
step accuracy :  0.6928
step loss :  0.11006863293647766
step : 2750/3206 
step accuracy :  0.7047272727272728
step loss :  0.10470756292343139
step : 3000/3206 
step accuracy :  0.715
step loss :  0.10059264588356018


/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([6, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


step : 3250/3206 
step accuracy :  0.7129230769230769
step loss :  0.0960506331370427
EPOCH 1
Train Accuracy :  0.722707423580786
Dev Accuracy :  0.8977556109725686
Train Mean loss :  0.09736885767167461
Dev Mean loss :  0.026813083381724178
----------------------------------------
step : 250/3206 
step accuracy :  0.896
step loss :  0.03632080411911011
step : 500/3206 
step accuracy :  0.874
step loss :  0.033589823365211484
step : 750/3206 
step accuracy :  0.8746666666666667
step loss :  0.03187296589215596
step : 1000/3206 
step accuracy :  0.856
step loss :  0.03722017794847488
step : 1250/3206 
step accuracy :  0.8552
step loss :  0.035757923078536986
step : 1500/3206 
step accuracy :  0.864
step loss :  0.03288213566939036
step : 1750/3206 
step accuracy :  0.8662857142857143
step loss :  0.03226460065160479
step : 2000/3206 
step accuracy :  0.874
step loss :  0.030667997792363167
step : 2250/3206 
step accuracy :  0.8835555555555555
step loss :  0.028205137146843803
step : 250

step : 1750/3206 
step accuracy :  0.9742857142857143
step loss :  0.002950656856291711
step : 2000/3206 
step accuracy :  0.9725
step loss :  0.0033909448380709363
step : 2250/3206 
step accuracy :  0.9733333333333334
step loss :  0.00326495023533385
step : 2500/3206 
step accuracy :  0.9756
step loss :  0.0030267449054153985
step : 2750/3206 
step accuracy :  0.9745454545454545
step loss :  0.0037246167677248278
step : 3000/3206 
step accuracy :  0.9723333333333334
step loss :  0.004208640179647773
step : 3250/3206 
step accuracy :  0.955076923076923
step loss :  0.004784383220501835
EPOCH 8
Train Accuracy :  0.9681846537741734
Dev Accuracy :  0.9152119700748129
Train Mean loss :  0.004850045373247337
Dev Mean loss :  0.022035124899814847
----------------------------------------
step : 250/3206 
step accuracy :  0.956
step loss :  0.014169363498687745
step : 500/3206 
step accuracy :  0.894
step loss :  0.03466157604474574
step : 750/3206 
step accuracy :  0.924
step loss :  0.023582

step : 250/3206 
step accuracy :  0.948
step loss :  0.01174955474305898
step : 500/3206 
step accuracy :  0.916
step loss :  0.01447324274899438
step : 750/3206 
step accuracy :  0.9253333333333333
step loss :  0.013887708537591
step : 1000/3206 
step accuracy :  0.942
step loss :  0.0110110623635519
step : 1250/3206 
step accuracy :  0.9504
step loss :  0.009766695440452895
step : 1500/3206 
step accuracy :  0.9553333333333334
step loss :  0.008410814322648851
step : 1750/3206 
step accuracy :  0.9554285714285714
step loss :  0.007768789598735956
step : 2000/3206 
step accuracy :  0.9605
step loss :  0.006869178633546625
step : 2250/3206 
step accuracy :  0.9564444444444444
step loss :  0.006828094915686532
step : 2500/3206 
step accuracy :  0.9588
step loss :  0.006544107676904218
step : 2750/3206 
step accuracy :  0.9607272727272728
step loss :  0.006187505626261203
step : 3000/3206 
step accuracy :  0.9616666666666667
step loss :  0.005987126713360321
step : 3250/3206 
step accura

In [21]:
def test_cbow(model, test_data, test_labels):
    acc = 0
    test_mean_loss = 0
    
    for x, labels in test_loader:
            
            preds = model(x)
            loss = loss_fn(preds, labels)
            test_mean_loss+=loss.item()
            for pred, label in zip(preds, labels):
                if((pred<0.5 and label==0) or (pred>0.5 and label == 1)):
                    acc+=1
    print("Test accuracy : ", acc/len(test_data))
    print("Test mean loss : ", test_mean_loss/len(test_data))

In [22]:
test_cbow(l_cbow, test_data, test_labels)

Test accuracy :  0.8830845771144279
Test mean loss :  0.05090465237252155


/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


## On plot les résultats

In [23]:
def plot_train_results(train_acc, dev_acc, train_loss, dev_loss, model_name):
    xaxis = [i for i in range(1,21)]
    acc_file_name = "acc_"+"_".join(model_name.split())+".png"
    loss_file_name = "loss_"+"_".join(model_name.split())+".png"
    
    plt.plot(xaxis,train_acc, label= "Train accuracy")
    plt.plot(xaxis,dev_acc, label="Dev accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.title("Accuracy on train and dev set over 20 epochs for a "+model_name)
    plt.savefig(acc_file_name)
    plt.show()
    
    plt.plot(train_loss, label= "Train loss")
    plt.plot(dev_loss, label="Dev loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Mean loss on train and dev set over 20 epochs for a "+model_name)
    plt.savefig(loss_file_name)
    plt.show()

In [24]:
plot_train_results(rnn_train_accuracies, rnn_dev_accuracies, rnn_train_losses, rnn_dev_losses, "GRU")

NameError: name 'rnn_train_accuracies' is not defined

# Un classifieur RNN classique (GRU) 

In [25]:
class GRU_classifier(nn.Module):
    def __init__(self, nb_cells, hidden_size, vocab_size, embedding_dim, rnn_dropout, bidirectional=False):
        super(GRU_classifier, self).__init__()
        
        self.nb_cells = nb_cells
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_size, nb_cells, batch_first = True, dropout=rnn_dropout,bidirectional=bidirectional)

        if bidirectional:    
            self.fc = nn.Linear(2*hidden_size, 1)
        else:
            self.fc = nn.Linear(hidden_size, 1)
        self.is_bidirectional = bidirectional 
        self.dropout = nn.Dropout(0.2)
        nn.init.xavier_uniform_(self.fc.weight.data)  # Xavier/Glorot init for tanh
        nn.init.zeros_(self.fc.bias.data)  # Xavier/Glorot init for tanh
        
            
    
    def forward(self, inputs, hidden):
        batch_size = inputs.size(0)
        embeds = self.embedding(inputs)
        rnn_out, hidden = self.gru(embeds, hidden)
        
        out = self.dropout(rnn_out)

        out = self.fc(out)
        out = th.sigmoid(out)
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        #hidden = th.Tensor(self.nb_cells, batch_size, self.hidden_size)
        if self.is_bidirectional:
            hidden = weight.new(2*self.nb_cells, batch_size, self.hidden_size).zero_()
        else:
            hidden = weight.new(self.nb_cells, batch_size, self.hidden_size).zero_()
        return hidden

# La même classe mais avec de l'attention
on implémente une version simplifiée de l'attention présentée dans ce papier : https://www.cs.cmu.edu/~hovy/papers/16HLT-hierarchical-attention-networks.pdf 

In [26]:
class GRU_with_attention_classifier(nn.Module):
    def __init__(self, nb_cells, hidden_size, vocab_size, embedding_dim, rnn_dropout, bidirectional=False):
        super(GRU_with_attention_classifier, self).__init__()
        
        self.nb_cells = nb_cells
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_size, nb_cells, batch_first = True, dropout=rnn_dropout,bidirectional=bidirectional)
        
        
            
        self.is_bidirectional = bidirectional 
        self.dropout = nn.Dropout(0.2)
        #nn.init.xavier_uniform_(self.fc.weight.data)  # Xavier/Glorot init for tanh
        #nn.init.zeros_(self.fc.bias.data)  # Xavier/Glorot init for tanh
        
        if bidirectional:    
            self.fc_attn1 = nn.Linear(2*hidden_size, 100)
        else:
            self.fc_attn1 = nn.Linear(hidden_size, 100)
            
        self.fc_attn2 = nn.Linear(100,1, bias=False)    
        self.fc_attn3 = nn.Linear(hidden_size, 1)
    
        self.softmax = nn.Softmax(dim=1)
        self.tanh = nn.Tanh()
        
    def forward(self, inputs, hidden):
        batch_size = inputs.size(0)
        embeds = self.embedding(inputs)
        rnn_out, hidden = self.gru(embeds, hidden)
        
        out = self.dropout(rnn_out)
        attention_weight = self.fc_attn1(out)
        attention_weight = self.tanh(attention_weight)
        attention_weight = self.softmax(self.fc_attn2(attention_weight))
        out = (out*attention_weight).sum(dim=1)
        out = self.fc_attn3(out)
        out = th.sigmoid(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        #hidden = th.Tensor(self.nb_cells, batch_size, self.hidden_size)
        if self.is_bidirectional:
            hidden = weight.new(2*self.nb_cells, batch_size, self.hidden_size).zero_()
        else:
            hidden = weight.new(self.nb_cells, batch_size, self.hidden_size).zero_()
        return hidden

In [29]:
##Hyper-paramètres
nb_cells = 1
hidden_size = 32
embedding_dim = 10
learning_rate = 1e-2
loss_fn = nn.BCELoss()
rnn_attention = GRU_with_attention_classifier(nb_cells, hidden_size, vocab_size, embedding_dim, 0.0, bidirectional=False)
rnn_no_attention = GRU_classifier(nb_cells, hidden_size, vocab_size, embedding_dim, 0.0, bidirectional=False)

rnn_train_accuracies = []
rnn_train_losses = []
rnn_dev_accuracies = []
rnn_dev_losses = []

print(m)

GRU_classifier(
  (embedding): Embedding(43464, 10)
  (gru): GRU(10, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (dropout): Dropout(p=0.2)
)


In [30]:
def train_rnn(model, batch_size,lr, max_epochs=20, conv=False):
    optim = th.optim.Adam(params=model.parameters(), lr=lr, weight_decay=1e-4)
    model.train()
    best_dict = model.state_dict()
    best_acc = 0
    for i in range(max_epochs):
        train_accuracy = 0
        dev_accuracy = 0
        train_mean_loss = 0
        dev_mean_loss = 0

        n = 0 

        for x, labels in train_loader:
            loc_batch_size = x.size(0)
            h = model.init_hidden(loc_batch_size)
            
            optim.zero_grad()
            h = h.data
            
            n+=loc_batch_size
            
            preds, _ = model(x, h)  
            loss = loss_fn(preds, labels)
            h = h.detach()
            train_mean_loss+=loss.item()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optim.step()

            for pred, label in zip(preds, labels):
                if((label==1 and pred>0.5) or(label==0 and pred<0.5)):
                    train_accuracy+=1
        
            if(n%250==0):
                print("step : {}/{} ".format(n, len(train_data)))
                print("step accuracy : ", train_accuracy/n)
                print("step loss : ", train_mean_loss/n)
                
        for x, labels in dev_loader:
            loc_batch_size = x.size(0)
            h = model.init_hidden(loc_batch_size)
            preds, _ = model(x, h)
            loss = loss_fn(preds, labels)

            dev_mean_loss+=loss.item()
            
            for pred, label in zip(preds, labels):
                if((label==1 and pred>0.5) or(label==0 and pred<0.5)):
                    dev_accuracy+=1
        
        if(i==0):
            best_dict=model.state_dict()
            best_acc=dev_accuracy/len(dev_data)
        else:
            if(best_acc<(dev_accuracy/len(dev_data))):
                best_acc=dev_accuracy/len(dev_data)
                best_dict=model.state_dict()
                print("new best acc")
        
        rnn_train_accuracies.append(train_accuracy/len(train_data))
        rnn_train_losses.append(train_mean_loss/len(train_data))
        rnn_dev_accuracies.append(dev_accuracy/len(dev_data))
        rnn_dev_losses.append(dev_mean_loss/len(dev_data))
        
        print("EPOCH {}".format(i+1))
        print("Train Accuracy : ",train_accuracy/len(train_data))
        print("Dev Accuracy : ",dev_accuracy/len(dev_data))
        print("Train Mean loss : ",train_mean_loss/len(train_data))
        print("Dev Mean loss : ",dev_mean_loss/len(dev_data))
        print("----------------------------------------")
        
#     model.load_state_dict(best_dict)
#     acc = test(model, test_data, conv=conv)
#     stat_dict[model_name][4].append(acc)
#     print("Accuracy on test data : ", acc)
#     return best_dict

In [31]:
train_rnn(rnn_no_attention, batch_size, learning_rate)

step : 250/3206 
step accuracy :  0.564
step loss :  0.013613412857055664
step : 500/3206 
step accuracy :  0.556
step loss :  0.013522427916526794
step : 750/3206 
step accuracy :  0.564
step loss :  0.013304168780644735
step : 1000/3206 
step accuracy :  0.572
step loss :  0.013119274318218231
step : 1250/3206 
step accuracy :  0.5768
step loss :  0.012984024286270142
step : 1500/3206 
step accuracy :  0.5733333333333334
step loss :  0.013030472556749979
step : 1750/3206 
step accuracy :  0.5834285714285714
step loss :  0.012899696111679078
step : 2000/3206 
step accuracy :  0.5905
step loss :  0.012812653452157974
step : 2250/3206 
step accuracy :  0.6004444444444444
step loss :  0.012655423561731974
step : 2500/3206 
step accuracy :  0.6084
step loss :  0.01257606635093689
step : 2750/3206 
step accuracy :  0.6098181818181818
step loss :  0.012489228226921775
step : 3000/3206 
step accuracy :  0.614
step loss :  0.012410133103529612
EPOCH 1
Train Accuracy :  0.6138490330630069
Dev 

step : 500/3206 
step accuracy :  0.996
step loss :  0.000263763225171715
step : 750/3206 
step accuracy :  0.9973333333333333
step loss :  0.00018347163079306483
step : 1000/3206 
step accuracy :  0.996
step loss :  0.0002499177198042162
step : 1250/3206 
step accuracy :  0.9968
step loss :  0.00022089294628240168
step : 1500/3206 
step accuracy :  0.996
step loss :  0.00024801165459211914
step : 1750/3206 
step accuracy :  0.9965714285714286
step loss :  0.000218919564670484
step : 2000/3206 
step accuracy :  0.9965
step loss :  0.00022575820091878996
step : 2250/3206 
step accuracy :  0.9964444444444445
step loss :  0.00022986728635927042
step : 2500/3206 
step accuracy :  0.9964
step loss :  0.00024912818800657986
step : 2750/3206 
step accuracy :  0.9967272727272727
step loss :  0.00023895882146263665
step : 3000/3206 
step accuracy :  0.997
step loss :  0.00022544688746954005
EPOCH 8
Train Accuracy :  0.9971927635683094
Dev Accuracy :  0.970074812967581
Train Mean loss :  0.00022

step : 250/3206 
step accuracy :  1.0
step loss :  3.793676663190126e-05
step : 500/3206 
step accuracy :  0.998
step loss :  0.0001808416345156729
step : 750/3206 
step accuracy :  0.996
step loss :  0.00020730547533215335
step : 1000/3206 
step accuracy :  0.997
step loss :  0.00016045785305323078
step : 1250/3206 
step accuracy :  0.9976
step loss :  0.0001434650121955201
step : 1500/3206 
step accuracy :  0.9973333333333333
step loss :  0.0001421678678598255
step : 1750/3206 
step accuracy :  0.9977142857142857
step loss :  0.0001239117648552305
step : 2000/3206 
step accuracy :  0.998
step loss :  0.00011092454350728076
step : 2250/3206 
step accuracy :  0.9977777777777778
step loss :  0.0001092712645056761
step : 2500/3206 
step accuracy :  0.998
step loss :  9.888845333480277e-05
step : 2750/3206 
step accuracy :  0.9981818181818182
step loss :  9.062682448314842e-05
step : 3000/3206 
step accuracy :  0.9983333333333333
step loss :  8.6871884787494e-05
EPOCH 15
Train Accuracy : 

In [32]:
train_rnn(rnn_attention, batch_size, learning_rate)

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


step : 250/3206 
step accuracy :  0.6
step loss :  0.013579703092575073
step : 500/3206 
step accuracy :  0.624
step loss :  0.012980732083320618
step : 750/3206 
step accuracy :  0.6133333333333333
step loss :  0.012932185649871825
step : 1000/3206 
step accuracy :  0.626
step loss :  0.012855867564678191
step : 1250/3206 
step accuracy :  0.6344
step loss :  0.012905344247817993
step : 1500/3206 
step accuracy :  0.6353333333333333
step loss :  0.0128362371524175
step : 1750/3206 
step accuracy :  0.6462857142857142
step loss :  0.012611775415284293
step : 2000/3206 
step accuracy :  0.6585
step loss :  0.012392570197582244
step : 2250/3206 
step accuracy :  0.6622222222222223
step loss :  0.012201269798808628
step : 2500/3206 
step accuracy :  0.6728
step loss :  0.011978852260112763
step : 2750/3206 
step accuracy :  0.6818181818181818
step loss :  0.011809399756518277
step : 3000/3206 
step accuracy :  0.6893333333333334
step loss :  0.011610003083944321


/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([6, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


EPOCH 1
Train Accuracy :  0.6996257018091079
Dev Accuracy :  0.8104738154613467
Train Mean loss :  0.01150424623221661
Dev Mean loss :  0.008726051964441737
----------------------------------------
step : 250/3206 
step accuracy :  0.8
step loss :  0.00853714644908905
step : 500/3206 
step accuracy :  0.844
step loss :  0.007428053796291351
step : 750/3206 
step accuracy :  0.8466666666666667
step loss :  0.007210622509320577
step : 1000/3206 
step accuracy :  0.836
step loss :  0.007268127828836441
step : 1250/3206 
step accuracy :  0.8272
step loss :  0.007631059670448304
step : 1500/3206 
step accuracy :  0.8266666666666667
step loss :  0.007622853457927704
step : 1750/3206 
step accuracy :  0.8422857142857143
step loss :  0.007284270874091557
step : 2000/3206 
step accuracy :  0.8515
step loss :  0.006912405125796795
step : 2250/3206 
step accuracy :  0.8613333333333333
step loss :  0.006490575522184372
step : 2500/3206 
step accuracy :  0.868
step loss :  0.006234349647164345
step

EPOCH 8
Train Accuracy :  0.9956331877729258
Dev Accuracy :  0.9775561097256857
Train Mean loss :  0.00026717572118042655
Dev Mean loss :  0.0017883492671949045
----------------------------------------
step : 250/3206 
step accuracy :  0.996
step loss :  0.0004792562834918499
step : 500/3206 
step accuracy :  0.992
step loss :  0.0006080778315663337
step : 750/3206 
step accuracy :  0.9933333333333333
step loss :  0.0005056683380777637
step : 1000/3206 
step accuracy :  0.994
step loss :  0.000543049116851762
step : 1250/3206 
step accuracy :  0.9912
step loss :  0.0006303133249282837
step : 1500/3206 
step accuracy :  0.9906666666666667
step loss :  0.0006245773173868656
step : 1750/3206 
step accuracy :  0.9902857142857143
step loss :  0.000610750210072313
step : 2000/3206 
step accuracy :  0.991
step loss :  0.0005838281895266846
step : 2250/3206 
step accuracy :  0.992
step loss :  0.0005334393088188437
step : 2500/3206 
step accuracy :  0.9924
step loss :  0.0005092883978039027
st

EPOCH 15
Train Accuracy :  0.9912663755458515
Dev Accuracy :  0.9775561097256857
Train Mean loss :  0.0004589382199720431
Dev Mean loss :  0.002460253946779254
----------------------------------------
step : 250/3206 
step accuracy :  0.992
step loss :  0.0006120183514431119
step : 500/3206 
step accuracy :  0.996
step loss :  0.0003309687586151995
step : 750/3206 
step accuracy :  0.9973333333333333
step loss :  0.00023675575448820987
step : 1000/3206 
step accuracy :  0.998
step loss :  0.000195984379213769
step : 1250/3206 
step accuracy :  0.9976
step loss :  0.00022136302578728645
step : 1500/3206 
step accuracy :  0.9973333333333333
step loss :  0.00024140787906556702
step : 1750/3206 
step accuracy :  0.9971428571428571
step loss :  0.00022788033536302725
step : 2000/3206 
step accuracy :  0.997
step loss :  0.0002148584415699588
step : 2250/3206 
step accuracy :  0.9973333333333333
step loss :  0.00019532515911851078
step : 2500/3206 
step accuracy :  0.9968
step loss :  0.0002

In [33]:
def test_rnn(model, test_data, test_labels):
    acc = 0
    test_mean_loss = 0
    
    for x, labels in test_loader:
            loc_batch_size = x.size(0)
            h = model.init_hidden(loc_batch_size)
            preds, _ = model(x, h)
            loss = loss_fn(preds, labels)
            test_mean_loss+=loss.item()
            for pred, label in zip(preds, labels):
                if((pred<0.5 and label==0) or (pred>0.5 and label == 1)):
                    acc+=1
    print("Test accuracy : ", acc/len(test_data))
    print("Test mean loss : ", test_mean_loss/len(test_data))

In [36]:
test_rnn(rnn_no_attention, test_data, test_labels)

Test accuracy :  0.9776119402985075
Test mean loss :  0.0021201462075994605


In [41]:
modified = 10*[0]
modified[0]="i think all that strife , it kind of helped me and both break down the barrier of caring what people thought about what we were doing , , mr ivey said the couple had twins , then lost one to a rare heart in 2010 ms price dealt with the loss , along with feelings of professional failure , by self with alcohol over the years , she and mr ivey a house s worth of prized gear and personal possessions to buy food , cover touring expenses and record midwest farmer s daughter at sun studio in memphis it kind of felt like there was the symbolism of it we are willing to put up the car , the wedding ring , she said we would rather have this album exist out here , even if nothing would have happened to it maybe 20 , 30 years down the road , or after i am dead , somebody would find it and it would become a cult thing like nick drake , you know \? photo things could have gone that way several indie and at least one major country label passed on "
modified[1]="paris \( reuters \) some of pablo picasso s greatest works go on display in paris this week , in an exhibition that charts a pivotal year in which he cemented his status as a titan of 20th century art picasso 1932 erotic year , chronicles the spanish artist s prolific creativity over those 12 months , referred to in the art world as his year of wonders picasso produced more than 300 works of art in 1932 and the exhibition at paris musee picasso features over 100 of them they include the masterpiece the dream a highly portrait of a sleeping marie walter , who was the artist s mistress at the time another highlight is a girl before a mirror , which also depicts walter we called 1932 an erotic year because picasso used as a driving force in his painting , exhibition curator told reuters the daughter of spanish painter pablo picasso , maya picasso is seen in front of her father 's paintings during a visit by french president emmanuel macron \( not pictured \) at the picasso 1932 erotic year exhibition at the picasso museum in paris , france , october"
modified[2]="french president emmanuel macron delivers his speech during a meeting at international food market in , near paris , france , october 11 , 2017 reuters francois mori pool paris \( reuters \) president emmanuel macron called for changes to france s food chain on wednesday to ensure that farmers , who have been hit by squeezed margins and a retail price war , are paid fairly macron said he supported a new type of contract , based on farmers production costs , which would require stronger producer organizations and a change in legislation the changes are part of a wide field to fork review promised by macron during his presidential campaign as he sought to appease farmers , an important constituency in french politics , who have also complained of excessive red tape last year a third of farmers earned less than 350 euros \( 403 \) a month , the agricultural mutual assistance association \( \) said , a third of the net minimum wage macron endorsed a proposal from the workshops to create a reversed contract starting from farmers , to food processors and to retailers this would ensure a better spread of added "

In [42]:
original = 10*[0]
original[0] = "i think all that strife , it kind of helped me and both break down the barrier of caring what people thought about what we were doing , , mr ivey said the couple had twins , then lost one to a rare heart in 2010 ms price dealt with the loss , along with feelings of professional failure , by self with alcohol over the years , she and mr ivey a house s worth of prized gear and personal possessions to buy food , cover touring expenses and record midwest farmer s daughter at sun studio in memphis it kind of felt like there was the symbolism of it we re willing to put up the car , the wedding ring , she said we would rather have this album exist out here , even if nothing would have happened to it maybe 20 , 30 years down the road , or after i m dead , somebody would find it and it would become a cult thing like nick drake , you know \? photo things could have gone that way several indie and at least one major country label passed on "
original[1]="paris \( reuters \) some of pablo picasso s greatest works go on display in paris this week , in an exhibition that charts a pivotal year in which he cemented his status as a titan of 20th century art picasso 1932 erotic year , chronicles the spanish artist s prolific creativity over those 12 months , referred to in the art world as his year of wonders picasso produced more than 300 works of art in 1932 and the exhibition at paris musee picasso features over 100 of them they include the masterpiece the dream a highly portrait of a sleeping marie walter , who was the artist s mistress at the time another highlight is girl before a mirror , which also depicts walter we called 1932 an erotic year because picasso used as a driving force in his painting , exhibition curator told reuters the daughter of spanish painter pablo picasso , maya picasso is seen in front of her father 's paintings during a visit by french president emmanuel macron \( not pictured \) at the picasso 1932 erotic year exhibition at the picasso museum in paris , france , october"
original[2]=""

In [57]:
def gru_infer(model, news):
    model.eval()
    idx = th.Tensor([word2idx[w] for w in news.split()]).long().view(1,-1)
    h0 = model.init_hidden(1)
    out,_=model(idx, h0)
    if(out>0.5):
        print("fake")
    else:
        print("false")
    return out
    

In [56]:
def cbow_infer(model, news):
    model.eval()
    idx = th.Tensor([word2idx[w] for w in news.split()]).long().view(1,-1)
    out=model(idx)
    if(out>0.5):
        print("fake")
    else:
        print("true")
    return out

In [59]:
i=1
print("cbow infer, original puis modifié")
print("original : ",cbow_infer(l_cbow, original[i]))
print("modifié : ",cbow_infer(l_cbow, modified[i]))
print(20*"-",'\n')

print("gru sans attention infer, original puis modifié")
print("original : ",gru_infer(rnn_no_attention, original[i]))
print("modifié : ",gru_infer(rnn_no_attention, modified[i]))
print(20*"-",'\n')

print("gru avec attention infer, original puis modifié")
print("original : ",gru_infer(rnn_attention, original[i]))
print("modifié : ",gru_infer(rnn_attention, modified[i]))


cbow infer, original puis modifié
fake
original :  tensor([[1.]], grad_fn=<SigmoidBackward>)
fake
modifié :  tensor([[1.]], grad_fn=<SigmoidBackward>)
-------------------- 

gru sans attention infer, original puis modifié
fake
original :  tensor([1.0000], grad_fn=<SelectBackward>)
fake
modifié :  tensor([1.0000], grad_fn=<SelectBackward>)
-------------------- 

gru avec attention infer, original puis modifié
fake
original :  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
fake
modifié :  tensor([[1.0000]], grad_fn=<SigmoidBackward>)


In [ ]:
tab[2]